Funciones JSON
===

**Preparacion**

In [1]:
import sqlite3

from IPython.core.magic import register_cell_magic, register_line_magic

conn = sqlite3.connect(":memory:")
cur = conn.cursor()


@register_line_magic
def sql(line):
    return cur.execute(line).fetchall()

**Funciones**

In [2]:
##
## json(X)
## verifica si su argumento es un string json valido y retorna 
## una version con los espacios en blanco removidos.
##
%sql SELECT json(' { "this" : "is", "a": [ "test" ] } ')

[('{"this":"is","a":["test"]}',)]

In [3]:
##
## json_array(X1, ...)
## recibe 0 o mas argumentos y retorna un string json
##
%sql SELECT json_array(1,2,'3',4) 

[('[1,2,"3",4]',)]

In [4]:
%sql SELECT json_array('[1,2]')

[('["[1,2]"]',)]

In [5]:
%sql SELECT json_array(json_array(1,2))

[('[[1,2]]',)]

In [6]:
%sql SELECT json_array(1,null,'3','[4,5]','{"six":7.7}')

[('[1,null,"3","[4,5]","{\\"six\\":7.7}"]',)]

In [7]:
%sql SELECT json_array(1,null,'3',json('[4,5]'),json('{"six":7.7}'))

[('[1,null,"3",[4,5],{"six":7.7}]',)]

In [8]:
##
## json_extract(X, P1, P2, ...)
##
%sql SELECT json_extract('{"a":2,"c":[4,5,{"f":7}]}', '$')

[('{"a":2,"c":[4,5,{"f":7}]}',)]

In [9]:
%sql SELECT json_extract('{"a":2,"c":[4,5,{"f":7}]}', '$.c')

[('[4,5,{"f":7}]',)]

In [10]:
%sql SELECT json_extract('{"a":2,"c":[4,5,{"f":7}]}', '$.c[2]')

[('{"f":7}',)]

In [11]:
%sql SELECT json_extract('{"a":2,"c":[4,5,{"f":7}]}', '$.c[2].f') 

[(7,)]

In [12]:
%sql SELECT json_extract('{"a":2,"c":[4,5],"f":7}','$.c','$.a')

[('[[4,5],2]',)]

In [13]:
%sql SELECT json_extract('{"a":2,"c":[4,5],"f":7}','$.c[#-1]')

[(5,)]

In [14]:
%sql SELECT json_extract('{"a":2,"c":[4,5,{"f":7}]}', '$.x') 

[(None,)]

In [15]:
%sql SELECT json_extract('{"a":2,"c":[4,5,{"f":7}]}', '$.x', '$.a')

[('[null,2]',)]

In [16]:
%sql SELECT json_extract('{"a":"xyz"}', '$.a')

[('xyz',)]

In [17]:
%sql SELECT json_extract('{"a":null}', '$.a')

[(None,)]

In [18]:
##
## Operadores -> y ->>
## Permiten extraer subcomponentes del json.
##
%sql SELECT '{"a":2,"c":[4,5,{"f":7}]}' -> '$'

[('{"a":2,"c":[4,5,{"f":7}]}',)]

In [19]:
%sql SELECT '{"a":2,"c":[4,5,{"f":7}]}' -> '$.c' 

[('[4,5,{"f":7}]',)]

In [20]:
%sql SELECT '{"a":2,"c":[4,5,{"f":7}]}' -> 'c'

[('[4,5,{"f":7}]',)]

In [21]:
%sql SELECT '{"a":2,"c":[4,5,{"f":7}]}' -> '$.c[2]'

[('{"f":7}',)]

In [22]:
%sql SELECT '{"a":2,"c":[4,5,{"f":7}]}' -> '$.c[2].f'

[('7',)]

In [23]:
%sql SELECT '{"a":2,"c":[4,5],"f":7}' -> '$.c[#-1]'

[('5',)]

In [24]:
%sql SELECT '{"a":2,"c":[4,5,{"f":7}]}' -> '$.x' 

[(None,)]

In [25]:
%sql SELECT '[11,22,33,44]' -> 3

[('44',)]

In [26]:
%sql SELECT '[11,22,33,44]' ->> 3

[(44,)]

In [27]:
%sql SELECT '{"a":"xyz"}' -> '$.a'

[('"xyz"',)]

In [28]:
%sql SELECT '{"a":"xyz"}' ->> '$.a'

[('xyz',)]

In [29]:
%sql SELECT '{"a":null}' -> '$.a'

[('null',)]

In [30]:
%sql SELECT '{"a":null}' ->> '$.a'

[(None,)]

In [31]:
##
## json_insert()
##
%sql SELECT json_insert('[1,2,3,4]','$[#]',99)

[('[1,2,3,4,99]',)]

In [32]:
%sql SELECT json_insert('[1,[2,3],4]','$[1][#]',99)

[('[1,[2,3,99],4]',)]

In [33]:
%sql SELECT json_insert('{"a":2,"c":4}', '$.a', 99)

[('{"a":2,"c":4}',)]

In [34]:
%sql SELECT json_insert('{"a":2,"c":4}', '$.e', 99) 

[('{"a":2,"c":4,"e":99}',)]

In [35]:
##
## json_replace()
##
%sql SELECT json_replace('{"a":2,"c":4}', '$.a', 99)

[('{"a":99,"c":4}',)]

In [36]:
%sql SELECT json_replace('{"a":2,"c":4}', '$.e', 99)

[('{"a":2,"c":4}',)]

In [37]:
%sql SELECT json_set('{"a":2,"c":4}', '$.a', 99)

[('{"a":99,"c":4}',)]

In [38]:
%sql SELECT json_set('{"a":2,"c":4}', '$.e', 99)

[('{"a":2,"c":4,"e":99}',)]

In [39]:
%sql SELECT json_set('{"a":2,"c":4}', '$.c', '[97,96]') 

[('{"a":2,"c":"[97,96]"}',)]

In [40]:
%sql SELECT json_set('{"a":2,"c":4}', '$.c', json('[97,96]'))

[('{"a":2,"c":[97,96]}',)]

In [41]:
%sql SELECT json_set('{"a":2,"c":4}', '$.c', json_array(97,96))

[('{"a":2,"c":[97,96]}',)]

In [42]:
##
## json_object()
## acepta 0 o un número par de argumentos y retorna una cadena json valida
##
%sql SELECT json_object('a',2,'c',4)

[('{"a":2,"c":4}',)]

In [43]:
%sql SELECT json_object('a',2,'c','{e:5}')

[('{"a":2,"c":"{e:5}"}',)]

In [44]:
%sql SELECT json_object('a',2,'c',json_object('e',5)) 

[('{"a":2,"c":{"e":5}}',)]

In [45]:
##
## json_remove(X, P, ...)
##
%sql SELECT json_remove('[0,1,2,3,4]','$[2]')

[('[0,1,3,4]',)]

In [46]:
%sql SELECT json_remove('[0,1,2,3,4]','$[2]','$[0]')

[('[1,3,4]',)]

In [47]:
%sql SELECT json_remove('[0,1,2,3,4]','$[0]','$[2]')

[('[1,2,4]',)]

In [48]:
%sql SELECT json_remove('[0,1,2,3,4]','$[#-1]','$[0]')

[('[1,2,3]',)]

In [49]:
%sql SELECT json_remove('{"x":25,"y":42}')

[('{"x":25,"y":42}',)]

In [50]:
%sql SELECT json_remove('{"x":25,"y":42}','$.z')

[('{"x":25,"y":42}',)]

In [51]:
%sql SELECT json_remove('{"x":25,"y":42}','$.y')

[('{"x":25}',)]

In [52]:
%sql SELECT json_remove('{"x":25,"y":42}','$') 

[(None,)]

In [53]:
##
## json_valid(X)
##
%sql SELECT json_valid('{"x":35}')

[(1,)]

In [54]:
%sql SELECT json_valid('{"x":35') 

[(0,)]